In [2]:
## LSTM model 
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import LSTM, Flatten, Dense
from tensorflow.keras.models import Sequential
import tensorflow.keras.backend as K
from tensorflow.keras.callbacks import EarlyStopping
from tqdm import tqdm

from Deep_learning.model import Model
from Deep_learning.Deep_tools import *

여기 경로 변경

In [3]:
data = pd.read_pickle("Data/ETF_final.pickle") # 파일 불러오는 경로 변경
data

,TLT,SHY,GLD,GSG,VNQ,BIL,DBA,IJS,QQQ,SPY,VWO
2007-05-31,86.38,80.07,65.54,41.200,78.95,91.60,26.35,40.87,47.21,153.32,43.66
2007-06-01,85.59,79.74,66.44,41.570,79.06,91.62,26.39,41.29,47.24,154.08,44.65
2007-06-04,85.98,79.77,66.54,42.100,79.38,91.64,26.23,41.35,47.38,154.10,44.51
2007-06-05,85.44,79.70,66.37,42.007,78.12,91.64,26.31,41.00,47.38,153.49,44.27
2007-06-06,85.40,79.80,66.41,42.030,77.59,91.68,26.11,40.51,46.88,151.84,43.53
...,...,...,...,...,...,...,...,...,...,...,...
2023-04-25,106.96,82.35,185.75,20.010,81.25,91.73,20.99,89.45,309.99,406.08,39.22
2023-04-26,105.82,82.25,184.74,19.600,80.47,91.75,21.02,88.66,311.87,404.36,39.49
2023-04-27,104.77,82.09,184.75,19.630,82.29,91.78,20.98,90.18,320.35,412.41,40.00
2023-04-28,106.46,82.18,184.80,19.910,83.30,91.78,20.95,91.35,322.56,415.93,40.24


In [4]:
# 요부분은 나중에 반복문 돌릴떄...
freq = "Q"
idx_list = pd.date_range("2008","2023", freq=freq)
len(idx_list)

60

In [5]:
# 파라미터 그리드 생성
from sklearn.model_selection import ParameterGrid

param_grid = {
    "hidden_layers":[2],
    "epochs":[100, 200, 300, 400, 500],
    "nodes" :[16, 32, 64, 80, 96, 112, 128],
    "lookbackwindow":[100,200,300,400]
}

# Create the parameter grid
grid = list(ParameterGrid(param_grid))

여기 경로 변경

In [8]:
import os

results = pd.DataFrame(columns = ['epochs', 'num_nodes', 'num_hidden_layers', 'look_back_window', 'sharpe_ratio'])
i = 0

#output_dir = '' # 여기 경로 파일 경로로 수정해야함 (결과 파일이 저장될 구글 드라이브 경로를 지정해야함)
#existing_files = os.listdir(output_dir)


for params in tqdm(grid[::-1]):
    print(params.values(), "Preprocessing.....")
    
    hidden_layer = params["hidden_layers"]
    epochs = params["epochs"]
    lookbackwindow = params["lookbackwindow"]
    nodes = params["nodes"]
    
    # Define Data Generater
    data_generator = split_time_series(data, lookbackwindow, freq = freq)
    
    # Weight 저장할 데이터프레임 (Reset DataFrame)
    weight_df = pd.DataFrame(columns=data.columns)
    
    for _ in range(len(idx_list)): 
        # Model 초기화   
        tf.keras.backend.clear_session()
        model = Model(epochs=epochs, num_nodes=nodes, num_hidden_layers=hidden_layer)

        # Data 불러오기
        sample_data = next(data_generator)
        idx = sample_data.index[-1]

        # Model 학습 및 결과 저장
        allocations = model.get_allocations(data=sample_data)
        weight_df.loc[idx] = allocations
    
    # Save Result
    pf_result, weight, sharpe_ratio = simulate_strategy(weight_df, data.pct_change(), fee_rate = 0.0015)
    results.loc[i] =[epochs, nodes, hidden_layer, lookbackwindow, sharpe_ratio]
    results.to_csv(f'result_dataframe_ryu_rev{i}.csv')
    i += 1

  0%|          | 0/140 [00:00<?, ?it/s]

dict_values([500, 2, 400, 128]) Preprocessing.....
Epoch 1/500
1/1 [==============================] - 2s 2s/step - loss: -0.0524
Epoch 2/500
1/1 [==============================] - 0s 85ms/step - loss: -0.0871
Epoch 3/500
1/1 [==============================] - 0s 91ms/step - loss: -0.1052
Epoch 4/500
1/1 [==============================] - 0s 90ms/step - loss: -0.1118
Epoch 5/500
1/1 [==============================] - 0s 90ms/step - loss: -0.1192
Epoch 6/500
1/1 [==============================] - 0s 96ms/step - loss: -0.1231
Epoch 7/500
1/1 [==============================] - 0s 90ms/step - loss: -0.1260
Epoch 8/500
1/1 [==============================] - 0s 104ms/step - loss: -0.1284
Epoch 9/500
1/1 [==============================] - 0s 97ms/step - loss: -0.1306
Epoch 10/500
1/1 [==============================] - 0s 87ms/step - loss: -0.1331
Epoch 11/500
1/1 [==============================] - 0s 110ms/step - loss: -0.1358
Epoch 12/500
1/1 [==============================] - 0s 106ms/step -

  0%|          | 0/140 [00:15<?, ?it/s]


KeyboardInterrupt: 